In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
minmax = lambda x: (min(x), max(x))
import matplotlib as mpl
COLOR = 'black'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR



def encode(series, n):
    plot = np.zeros((n,n))
    series = series[::len(series)//n][:n]
    series/=series.max()
    disc = (series*(n-1)).astype(int)
    for i in range(n):
        miny, maxy = minmax(disc[i:i+2])
        plot[miny:maxy+1,i]=1
    plot+=np.random.rand(plot.shape[0], plot.shape[1])*0.5
    return plot
x = np.sin(np.linspace(0, 2*np.pi, 64))**2
# set figsize to 400x300 pixels
plt.figure(figsize=(4,3))

ax1, ax2 = plt.subplot(121), plt.subplot(122)
# disable ticks for ax1 and ax2
ax1.set_xticks([])
ax1.set_yticks([])
ax1.plot(x)
ax2.matshow(encode(x, 64), origin="lower", cmap="gray")
ax2.set_xticks([])
ax2.set_yticks([])

# save fig 
plt.savefig("plot-coding.png", dpi=100, bbox_inches="tight")

In [ ]:
import pandas as pd
prices = pd.read_csv("/datasets/eur_usd/eurusd_minute.csv")['BidClose'].values
prices.shape

In [ ]:
from torch.utils.data import Dataset
import torch

class PlotDataset(Dataset):
    def __init__(self, prices, n):
        values = np.array(prices)
        values = values[:len(values)//n*n]
        values = values.reshape(-1,n)
        values -= values.min(axis=1).reshape(-1,1)
        plots = np.zeros((len(prices)//n, n, n))
        for i in range(len(values)):
            plots[i]= encode(values[i], n)

        self.x = torch.from_numpy(plots).float().view(-1,1,n,n)


    def __len__(self):
        return self.x.size(0) 

    def __getitem__(self, idx):
        return self.x[idx]

dataset = PlotDataset(prices, 64)
plt.matshow(dataset[0][0], origin="lower", cmap="gray")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type=='cuda':
    !nvidia-smi

In [ ]:
from src.models import Discriminator, Generator
from src.trainer import WGanGpTrainer
import torch.optim as optim
from torch.utils.data import DataLoader

d = Discriminator()
g = Generator()


trainer = WGanGpTrainer(
    generator=g,
    critic=d,
    gen_optimizer=optim.Adam(g.parameters(), lr=1e-4, betas=(0.0, 0.9)),
    critic_optimizer=optim.Adam(d.parameters(), lr=1e-4, betas=(0.0, 0.9)),
    latent_dimension=100,
    device=device,
    model_dir="/storage/plot_model2",
    write_dir="/storage/plot_experiment2",
    checkpoint_interval=5,
)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
trainer.train(train_loader, 300)

In [ ]:
import os
import torch
from src.models import Generator
import matplotlib.pyplot as plt

g=Generator()
for i in os.listdir("/storage/plot_model/checkpoints"):
    path = f"/storage/plot_model/checkpoints/{i}/g.pt"
    g.load_state_dict(torch.load(path))
    noise = torch.rand(1,100)
    mat=g(noise)[0][0].detach().numpy()
    plt.matshow(mat)

In [ ]:
from torch.utils.data import Dataset
import torch

import pandas as pd
prices = pd.read_csv("/datasets/eur_usd/eurusd_minute.csv")['BidClose'].values
prices.shape

class Plot2TsDataset(Dataset):
    def __init__(self, prices, n):
        values = np.array(prices)
        values = values[:len(values)//n*n]
        values = values.reshape(-1,n)
        values -= values.min(axis=1).reshape(-1,1)
        plots = np.zeros((len(prices)//n, n, n))
        for i in range(len(values)):
            plots[i]= encode(values[i], n)

        self.x = torch.from_numpy(plots).float().view(-1,1,n,n)
        self.y = torch.from_numpy(values).float().view(-1,1,n)


    def __len__(self):
        return self.x.size(0) 

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

p2ts_dataset = PlotDataset(prices, 64)

In [ ]:
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type=='cuda':
    !nvidia-smi

class Decoder(nn.Module):
    def __init__(self, channels=1, sigmoid=False) -> None:
        super(Discriminator, self).__init__()
        layers = [
            nn.Conv2d(channels, 64, (4, 4), (2, 2), (1, 1), bias=True),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(64, 128, (4, 4), (2, 2), (1, 1), bias=False),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(128, 256, (4, 4), (2, 2), (1, 1), bias=False),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(256, 512, (4, 4), (2, 2), (1, 1), bias=False),
            nn.LeakyReLU(0.2, True),

            nn.Conv2d(512, 64, (4, 4), (1, 1), (0, 0), bias=True),
        ]
        if sigmoid:
            layers.append(nn.Sigmoid())

        self.main = nn.Sequential(*layers)

            
    def forward(self, x: Tensor) -> Tensor:
        out = self.main(x)
        out = torch.flatten(out, 2)
        return out

In [ ]:
def train(model, train_loader, epochs=10, lr=1e-3, device=device):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    losses = []
    for epoch in range(epochs):
        for i, (x,y) in enumerate(tqdm(train_loader)):
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            if i%100==0:
                losses.append(loss.item())
    return losses

train_loader = DataLoader(p2ts_dataset, batch_size=32, shuffle=True)
decoder = Decoder()
losses = train(decoder, train_loader, epochs=1)

plt.plot(losses)